In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [2]:
# set the parameters to run the script
data_folder_path = r'C:\Users\romai\Desktop\Freezing_dev'
# data_folder_path = r'C:\Users\romai\Documents\illustrators\pathology_protocol\imgs\effect_freezing'
directories = [data_folder_path]
calib_directory = r'C:\Users\romai\Desktop\calib'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [3]:
time_base = 'T1_'
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path, time_base)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:34<00:00,  1.91s/it]


In [4]:
base_dirs

['C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-11_T_FRE-FR-T1_FR_S1_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-06-30_T_FRE-FR-T1_FR_S2_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-06-30_T_FRE-FR-T1_FR_S3_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-06-30_T_FRE-FR-T1_FR_S4_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-06-30_T_FRE-FR-T1_FR_S5_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-06-30_T_FRE-FR-T1_FR_S6_1']

## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [5]:
import time

# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    propagation_lists = {}
    
    for tissue_type in tissue_types:
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_lists[tissue_type] = selection_of_ROIs.square_selection(path_folder_temp, path_folder, WM, 
                                                                            tissue_type, path_align_folder)
    # 3. actually do the alignment
    propagation_lists = automatic_ROI_propagation.generate_combined_mask(propagation_lists, path_align_folder)
    automatic_ROI_propagation.do_alignment(path_align_folder)
    output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
    
    # 4. and collect the data after propagation
    for tissue_type in tissue_types:
        WM = tissue_type == 'WM'
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_lists[tissue_type], 
                                                            output_folders, data_folder_path, time_base)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Folder already exists, trying again...
Folder already exists, trying again...


 17%|█████████████▊                                                                     | 1/6 [02:43<13:38, 163.63s/it]

Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...


 33%|███████████████████████████▋                                                       | 2/6 [05:22<10:44, 161.08s/it]

...

 67%|███████████████████████████████████████████████████████▎                           | 4/6 [10:40<05:18, 159.29s/it]

Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...


 83%|█████████████████████████████████████████████████████████████████████▏             | 5/6 [13:13<02:36, 156.87s/it]

Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [15:49<00:00, 158.33s/it]


## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [6]:
from freezingeffect import parameter_comparaison

In [29]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_10_align',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_11_align',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_12_align',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_13_align',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_14_align',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_5_align',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_6_align',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_7_align',
 'C:\\Users\\romai\

#### 4.2. Get the data for the measurement

In [32]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['WM'][list(data_fixation['WM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,150.110170,18.125541,147.640449,150.110170,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.934641,0.010353,0.932911,0.934587,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.027084,0.007280,0.020408,0.026293,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,32.044758,14.179427,26.202532,30.034710,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,158.618573,29.165514,159.775281,158.618573,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.943704,0.009130,0.932911,0.943621,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.038279,0.008160,0.030612,0.037973,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,19.000160,11.239742,12.531646,17.175164,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,146.615768,17.150455,147.640449,146.615768,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.938433,0.009751,0.932911,0.937844,None


In [33]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [35]:
data_fixation['WM'][list(data_fixation['WM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,150.110170,18.125541,147.640449,150.110170,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.934641,0.010353,0.932911,0.934587,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.027084,0.007280,0.020408,0.026293,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,32.044758,14.179427,26.202532,30.034710,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,158.618573,29.165514,159.775281,158.618573,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.943704,0.009130,0.932911,0.943621,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.038279,0.008160,0.030612,0.037973,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,19.000160,11.239742,12.531646,17.175164,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,146.615768,17.150455,147.640449,146.615768,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.938433,0.009751,0.932911,0.937844,None


In [42]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1', 'T2']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [44]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,30.034710,12.394438,12.909636,14.394906,10.379600,9.381300,11.296512,12.701826,9.425515,10.187369,...,13.605579,15.657726,15.027348,14.271038,18.462742,11.566009,13.930787,13.317727,12.084212,16.065332
1,17.175164,13.342462,12.788840,12.638395,10.359061,14.566736,18.114228,17.867097,13.609657,14.584848,...,13.555640,15.099033,14.731747,16.573545,12.343137,14.979855,15.985965,15.808187,13.820491,19.894670
2,25.513813,12.992534,15.658937,12.384673,11.340934,15.550750,24.239319,22.052117,14.798117,15.010945,...,13.475549,16.355733,10.813123,14.460345,13.196751,NaN,17.273392,17.212131,10.539852,16.926095


In [45]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,1.748729,0.928947,1.009445,1.138982,1.001983,0.644022,0.623626,0.710906,0.692561,0.698490,...,1.003684,1.037002,1.020066,0.861073,1.495790,0.772104,0.871439,0.842458,0.874369,0.807519
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.485506,0.973773,1.224422,0.979925,1.094784,1.067552,1.338137,1.234231,1.087325,1.029215,...,0.994092,1.083231,0.734001,0.872496,1.069157,NaN,1.080535,1.088811,0.762625,0.850785


#### 4.3. Statistical analysis

In [46]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [47]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value    n_1    n_2  before/GM  \
parameter      data_type time                                                   
azimuth        GM        T0        0.0  2.327063e-53  150.0  113.0        0.0   
                         T1    11250.0  1.000000e+00  150.0  150.0        0.0   
                         T2        0.0  1.953051e-55  150.0  130.0        0.0   
               WM        T0        0.0  1.010763e-54  150.0  124.0        0.0   
                         T1    11250.0  1.000000e+00  150.0  150.0        0.0   
                         T2        0.0  2.560486e-55  150.0  129.0        0.0   
depolarization GM        T0    10650.0  7.956941e-05  150.0  113.0        1.0   
                         T1    11250.0  1.000000e+00  150.0  150.0        1.0   
                         T2      750.0  1.686747e-47  150.0  130.0        1.0   
               WM        T0    15000.0  1.320791e-21  150.0  124.0        1.0   
                         T1    11250.0  1.000000e+00  150.0  150.0        1.0   
                         T2    14850.0  5.319187e-17  150.0  129.0        1.0   
retardance     GM        T0     4200.0  8.750909e-15  150.0  113.0        1.0   
                         T1    11250.0  1.000000e+00  150.0  150.0        1.0   
                         T2    15450.0  4.790240e-20  150.0  130.0        1.0   
               WM        T0     9600.0  6.158513e-01  150.0  124.0        1.0   
                         T1    11250.0  1.000000e+00  150.0  150.0        1.0   
                         T2    11400.0  5.228659e-03  150.0  129.0        1.0   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0              0.0  28.144230     22.757039  
                         T1              0.0   0.000000      0.000000  
                         T2              0.0  21.278400     21.438884  
               WM        T0              0.0  30.611568     23.353734  
                         T1              0.0   0.000000      0.000000  
                         T2              0.0  13.892018     15.495027  
depolarization GM        T0              0.0   0.981337      0.041626  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   1.074696      0.047261  
               WM        T0              0.0   0.991638      0.013371  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   0.996127      0.009134  
retardance     GM        T0              0.0   1.487476      0.651035  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   0.846210      0.219117  
               WM        T0              0.0   1.043262      0.339563  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   0.985694      0.171888

In [48]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [49]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,NaN,1.150376,1.762616,1.512431,1.609404,NaN,2.581769,1.788474,NaN,1.282906,...,1.003684,1.037002,1.020066,0.861073,1.495790,0.772104,0.871439,0.842458,0.874369,0.807519
1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,NaN,0.666416,0.602399,0.682510,0.881979,0.463415,1.034169,1.481431,0.837464,0.783116,...,0.994092,1.083231,0.734001,0.872496,1.069157,NaN,1.080535,1.088811,0.762625,0.850785


In [50]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))